# Universidade Federal da Paraíba - UFPB

# Programa de Pós-Graduação em Economia - PPGE

# Introdução à Data Science 

## Modelos de Machine Learning - Naives Bayes e k-Nearest Neighbour (kNN)

### O que é Machine Learning ?

Machine Learning é um campo da computação que envolve o processo de aprendizagem de inteligência artificial com o intuito de analisar dados, identificar padrões, prever informações e, sobretudo, interagir com atividades humanas.

"Machine Learning é o campo de estudo que fornece aos computadores a habilidade de aprender sem serem explicitamente programados" Arthur L. Samuel, 1959


### Para quê se usa Machine Learning ? 

O uso de algoritmos de machine learning está inserida na vida cotidiana, mas pouco se faz notar. Filtragem de Spams na caixa de e-mail, recomendação de músicas e filmes nos aplicativos de streaming e reconhecimento facial são alguns poucos exemplos do vasto uso de algoritmos de ML no dia-a-dia. 


### Quais são os tipos de sistemas de Machine Learning que existem? 
São muitos os tipos de sistemas de Machine Learning e os critérios que definem um sistema não são exclusivos.
No entanto, pode-se ressaltar dois tipos fundamentais:

1) **Aprendizagem Supervisionada**: Na aprendizagem supervisionada, os dados que serão utilizados no algoritmo já
incluem os *labels*. Algumas tarefas comuns para esse tipo de aprendizagem são:

        1.1) Classificação: Classificar uma variável chamada de classe (geralmente ditocômica) baseado em outras características. Um exemplo clássico é classificar se um email é um spam ou não;
        
        1.2) Previsão: Prever um valor númerico de uma variável dado um conjunto de preditores. Prever o preço de um carro com base na quilometragem, marca, ano de fabricação, etc. é um exemplo de tarefa de previsão.
        
2) **Aprendizagem Não-Supervisionada**: Na aprendizagem não-supervisionada, os dados não são rotulados. Uma maneira de olhar é imaginar que o sistema tem que aprender sem um professor. Aqui, a atividade mais comum é *clustering* (agrupamento) de dados.

## Algoritmo Naive-Bayes

O **algoritmo Naive-Bayes (NB)** é um algoritmo de aprendizagem supervisionada de classificação. É utilizado, principalmente, em atividades de filtragem de spam, separação de documentos, etc. O Naive-Bayes faz uso de uma aborgadem probabilística, fundamentada no Teorema de Bayes. 

O NB é comumente tratado como um algoritmo ML de entrada, pela sua facilidade de entendimento do método, implementação do algoritmo e interpretação dos resultados. Ele também é considerado um algoritmo rápido, devido ao fato de que seus cálculos restringem-se ao cálculo de probabilidades *a priori* e *a posteriori*, que não exigem muito tempo computacional. Por fim, o NB é capaz de obter uma boa precisão nas classificações em bases pequenas, porém, para bases grandes (>1 milhão de linhas), existem algoritmos que performam melhor. 

Podemos avaliar as características do algoritmo Naive-Bayes utilizando um exemplo clássico: Decidir se devemos conceber ou não um empréstimo ao um indivíduo, baseado em um conjunto de características (preditores). 

In [69]:
# Pacotes necessários
import pandas as pd
import os

In [70]:
os.getcwd()
os.chdir('/home/pedro/Área de Trabalho/PPGE/Introdução a Data Science/apresentacao')

In [71]:
# Abrindo a base
base = pd.read_csv('credit-data.csv')
# Resumo estatístico
base.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


In [72]:
# Como temos idade negativa, podemoos preencher as linhas com idade negativa com a média (sem idades negativas)
MediaSemIdadeNegativa = base['age'][base.age > 0].mean()
base.loc[base.age < 0, 'age'] = MediaSemIdadeNegativa
base.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.927700,4444.369695,0.141500
std,577.494589,14326.327119,13.261825,3045.410024,0.348624
min,1.000000,20014.489470,18.055189,1.377630,0.000000
25%,500.750000,32796.459717,29.072097,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


In [73]:
# Para conferir, pela contagem, quee a coluna 'Age' tem valores 'nan' ou 'missing'
base.loc[pd.isnull(base['age'])]

# Substituindo esses valores pela a média (sem NaN)
import numpy as np
MediaSemNaN = np.nanmean(base['age'])
base.loc[pd.isnull(base['age']), 'age'] = MediaSemNaN
base.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.927700,4444.369695,0.141500
std,577.494589,14326.327119,13.251870,3045.410024,0.348624
min,1.000000,20014.489470,18.055189,1.377630,0.000000
25%,500.750000,32796.459717,29.102161,1939.708847,0.000000
50%,1000.500000,45789.117313,41.300710,3974.719419,0.000000
75%,1500.250000,57791.281668,52.582340,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


In [74]:
# Temos que dividir em regressores e regressando
X = base.iloc[:, 1:4].values
y = base.iloc[:,4].values

In [75]:
# Temos que dividir as bases em treino e teste.
from sklearn.model_selection import train_test_split
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=10)

In [76]:
# Importando o método Naive Bayes
from sklearn.naive_bayes import BernoulliNB
NB = BernoulliNB()

In [77]:
# Treinamos o algoritmo fornecendo as bases de treino
NB.fit(X_treino,y_treino)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [79]:
# Agora, dado que o algoritmo foir treinado, podemos fazer nossa previsão
y_treino_pred = NB.predict(X_teste)

In [89]:
# Para avaliar se a performance do algoritmo foi satisfatória, devemos verificar algumas métricas
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix

acuracia = accuracy_score(y_treino_pred, y_teste)
precisao = precision_score(y_treino_pred, y_teste)

print('Acurácia:', acuracia*100, '%')
print('Precisão:', precisao*100, '%')

Acurácia: 85.5 %
Precisão: 0.0 %


In [90]:
# Matriz de Confusão
confusion_matrix(y_treino_pred, y_teste)

array([[342,  58],
       [  0,   0]])